# What is Docker?

Lets download and find out ;)

## 1
[Download Docker](https://download.docker.com/win/stable/InstallDocker.msi)
and install then skip to step 2.

For Docker on Windows see [How to install Docker on Windows?](~/How To install Docker on Windows?.ipynb)

In [6]:
$$.async();
var path = require('path');
if (process.platform === 'win32') {
    var notebook = path.join(process.cwd(), 'How to install Docker on Windows.ipynb');
    runNotebook(notebook, () => {
        $$.done('executed notebook ' + notebook);
    });
} else if (process.platform === 'darwin') {
    var notebook = path.join(process.cwd(), 'How to install Docker on Mac.ipynb');
    runNotebook(notebook, () => {
        $$.done('executed notebook ' + notebook);
    });
} else {
    $$.done('docker not installed');
}

ReferenceError: runNotebook is not defined

Create a Docker container with some dev tools:
* selenium (xvfb, x11vnc, novnc, mocha)
* source code (git, act.ecommerce, angular-cli)

Start from the selenium-chrome-debug image

In [1]:
$$.async();
var path = require('path');
var dockerfilePath = path.join(process.cwd(), 'selenium-act', 'Dockerfile');
var fs = require('fs');

var aptPart = `
FROM selenium/standalone-chrome-debug

RUN apt-get -qq update
RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make 
RUN wget -O - https://deb.nodesource.com/setup_7.x | bash
RUN apt-get install -y nodejs
RUN nodejs -v

`;

var nodePart = `

ENV NODE_TLS_REJECT_UNAUTHORIZED 0
RUN npm install -g live-server babel-cli concurrently node-gyp nativescript@latest

COPY act.ecommerce /home/seluser/act.ecommerce
COPY selenium /home/seluser/selenium
ADD novnc /home/seluser/novnc

WORKDIR /home/seluser/act.ecommerce
RUN npm install
RUN npm run build

WORKDIR /home/seluser/selenium/ecommerce
RUN npm install

WORKDIR /home/seluser/
# modify the entry_point from selenium-docker to run our extra services
RUN dos2unix /home/seluser/novnc/utils/launch.sh
RUN sed -i '/wait \$NODE_PID/i/home/seluser/novnc/utils/launch.sh --vnc localhost:5900 &' /opt/bin/entry_point.sh
RUN sed -i '/wait \$NODE_PID/inpm --prefix /home/seluser/selenium/ecommerce run static &' /opt/bin/entry_point.sh

EXPOSE 4200
EXPOSE 4444
EXPOSE 3000

`;
var output = aptPart + nodePart;
var http = require('http');
// check if there is a local-npm server running on Brian's machine
http.get('http://SCO-100001269.ad.swiftpage.com:5080', (r) => {
    if (r.statusCode === 200) {
        console.log('local-npm found!');
        output = aptPart + `
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set registry http://sco-100001269.ad.swiftpage.com:5080
RUN npm config set strict-ssl false
` + nodePart;
    }
    fs.writeFile(dockerfilePath, output, function () {
        $$.done('saved Dockerfile');
    });
}).on('error', (e) => {
    fs.writeFile(dockerfilePath, output, function () {
        $$.done('saved Dockerfile');
    });
});



{ Error: getaddrinfo ENOTFOUND sco-100001269.ad.swiftpage.com sco-100001269.ad.swiftpage.com:5080
    at errnoException (dns.js:28:10)
    at GetAddrInfoReqWrap.onlookup [as oncomplete] (dns.js:76:26)
  code: 'ENOTFOUND',
  errno: 'ENOTFOUND',
  syscall: 'getaddrinfo',
  hostname: 'sco-100001269.ad.swiftpage.com',
  host: 'sco-100001269.ad.swiftpage.com',
  port: '5080' }


'saved Dockerfile'

## 2

Create a Docker file and project directory.

In [ ]:
$$.async();
var path = require('path');
var notebook = path.join(process.cwd(), 'What repositories do we need.ipynb');
runNotebook(notebook, () => {
    $$.done('executed notebook ' + notebook);
});

## 3
Build the Docker image

In [2]:
$$.async();
var exec = require('child_process').exec;
var path = require('path');
var seleniumPath = path.join(process.cwd(), 'selenium-act');
var buildCmd = exec('docker build -t act-selenium ' + seleniumPath, () => {
    var imagesCmd = exec('docker images', () => {
        $$.done('built Docker container')
    });
    imagesCmd.stdout.on('data', (d) => console.log(d));
    imagesCmd.stderr.on('data', (d) => console.log(d));
});
buildCmd.stdout.on('data', (d) => console.log(d));
buildCmd.stderr.on('data', (d) => console.log(d));





Step 1/23 : FROM selenium/standalone-chrome-debug

latest: Pulling from selenium/standalone-chrome-debug

c62795f78da9: Pulling fs layer

d4fceeeb758e: Pulling fs layer

5c9125a401ae: Pulling fs layer

0062f774e994: Pulling fs layer

6b33fd031fac: Pulling fs layer

73ed4702f19f: Pulling fs layer

0062f774e994: Waiting

6b33fd031fac: Waiting

f39678fd73a8: Pulling fs layer

11d1654494ca: Pulling fs layer

73ed4702f19f: Waiting

662143052064: Pulling fs layer

6afa6779fbcb: Pulling fs layer

e4f2b7486a1f: Pulling fs layer
11d1654494ca: Waiting
a344a8e6c06b: Pulling fs layer
5dbbc312b92f: Pulling fs layer
06fe993c8974: Pulling fs layer
662143052064: Waiting
6afa6779fbcb: Waiting

e4f2b7486a1f: Waiting
a344a8e6c06b: Waiting
5dbbc312b92f: Waiting
01ba5fdc80c3: Pulling fs layer
06fe993c8974: Waiting
5b416f114ad6: Pulling fs layer

01ba5fdc80c3: Waiting
f68d2feb2e62: Pulling fs layer
f3a6e677d61d: Pulling fs layer
7a30b1c68862: Pulling fs layer
5b416f114ad6: Waiting
3dbb16382933: Pulling fs

'built Docker container'

## 4

Run your Docker container

In [5]:
$$.async();
var exec = require('child_process').exec;
console.log('stopping current Docker');
var stopCmd = exec('docker stop act-selenium', () => {
    console.log('remove current Docker');
    var rmCmd = exec('docker rm act-selenium', () => {
        console.log('running new Docker');
        var runCmd = exec('docker run --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 act-selenium', () => {
            console.log('checking new Docker is running');
            var psCmd = exec('docker ps', () => {
                // wait a few seconds for the container services to start
                setTimeout(() => {
                    $$.done('launched Docker container');
                }, 6000);
            });
            psCmd.stdout.on('data', (d) => console.log(d));
            psCmd.stderr.on('data', (d) => console.log(d));
        });
        runCmd.stdout.on('data', (d) => console.log(d));
        runCmd.stderr.on('data', (d) => console.log(d));
    });
    rmCmd.stdout.on('data', (d) => console.log(d));
    rmCmd.stderr.on('data', (d) => console.log(d));
});
stopCmd.stdout.on('data', (d) => console.log(d));
stopCmd.stderr.on('data', (d) => console.log(d));

